In [1]:
from IPython.display import Image
from IPython.display import display, HTML
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from fboost import outlier_iqr, DataPreparator, FeatureBoosterRegressor
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LinearRegression, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from ffx import FFXRegressor
import tensorflow as tf
from tensorflow import keras

from keras.optimizers import Adam
from keras.layers import Lambda
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Conv1D, MaxPooling1D, concatenate, Conv2D, Dropout

from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

import warnings
warnings.filterwarnings('ignore')

# Set a seed value
seed_value= 0
# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)
# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

In [2]:
data = pd.read_csv('../data/raw/alldata.csv')
data.columns = ['familia','indice_fam','subposicion_1','subposicion_2','subposicion_3','subposicion_4', 'energia']
mask = (data[['subposicion_1','subposicion_2','subposicion_3','subposicion_4']] == 0).sum(axis=1) > 1
lista_train = data.loc[mask,['subposicion_1','subposicion_2','subposicion_3','subposicion_4','energia']]
lista_test = data.loc[~mask,['subposicion_1','subposicion_2','subposicion_3','subposicion_4','energia']]

X_train = lista_train.drop(['energia'], axis=1).astype(str)
X_test = lista_test.drop(['energia'], axis=1).astype(str)

y_test = lista_test['energia']
y_train = lista_train['energia']

X_train = X_train.astype(str)
X_train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 170 entries, 0 to 1269
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   subposicion_1  170 non-null    object
 1   subposicion_2  170 non-null    object
 2   subposicion_3  170 non-null    object
 3   subposicion_4  170 non-null    object
dtypes: object(4)
memory usage: 6.6+ KB


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor


# Specify your dictionary of words (customize n here)
dictionary = ['0','1','2','3','4','5']

X_train['sentence'] = X_train.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
X_test['sentence'] = X_test.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

X_train_array = np.array(X_train['sentence'])
X_test_array = np.array(X_test['sentence'])

In [4]:
X_train_array.replace('0','zero', regex=True)
X_train_array

AttributeError: 'numpy.ndarray' object has no attribute 'replace'

In [ ]:
# Initialize TfidfVectorizer with your dictionary
# TfidfVectorizer will convert sentences into vectors of length n
vectorizer = TfidfVectorizer()

# Transform the sentences in train and test data into vectors
X_train_vec = vectorizer.fit_transform(X_train_array)
X_test_vec = vectorizer.transform(X_test_array)

train_transformed = pd.DataFrame(X_train_vec.toarray())

train_transformed

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:


# Build a RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")


ValueError: Found input variables with inconsistent numbers of samples: [4, 170]